In [1]:
import numpy as np

In [ ]:
n = 10
width = 1.
depth = 0.1
spacing = 1/10

volume = depth*width*width
area = depth*width

x_p = np.linspace(0,1,n+1)
centroids = np.linspace(spacing/2,1-spacing/2,n)

grid = np.stack(np.meshgrid(centroids,centroids))



(2, 10, 10)